In [1]:
from platform import python_version
print(python_version())

3.6.12


In [2]:
import covasim as cv
print(cv.__version__)
cv.options.set(verbose=0) #no print output
# help(cv.options.set)
import pandas as pd
from numpy import mean

import plotly.express as px
import plotly.graph_objects as go

Covasim 3.0.5 (2021-05-28) — © 2021 by IDM
3.0.5


In [3]:
# saved as helper_stats.py for the app

# population data - derived from census 2019 estimate, also same as data used by covidactnow
population_dict = {'Alabama': 4903185,
 'Alaska': 731545,
 'Arizona': 7278717,
 'Arkansas': 3017804,
 'California': 39512223,
 'Colorado': 5758736,
 'Connecticut': 3565287,
 'Delaware': 973764,
 'District of Columbia': 705749,
 'Florida': 21477737,
 'Georgia': 10617423,
 'Hawaii': 1415872,
 'Idaho': 1787065,
 'Illinois': 12671821,
 'Indiana': 6732219,
 'Iowa': 3155070,
 'Kansas': 2913314,
 'Kentucky': 4467673,
 'Louisiana': 4648794,
 'Maine': 1344212,
 'Maryland': 6045680,
 'Massachusetts': 6892503,
 'Michigan': 9986857,
 'Minnesota': 5639632,
 'Mississippi': 2976149,
 'Missouri': 6137428,
 'Montana': 1068778,
 'Nebraska': 1934408,
 'Nevada': 3080156,
 'New Hampshire': 1359711,
 'New Jersey': 8882190,
 'New Mexico': 2096829,
 'New York': 19453561,
 'North Carolina': 10488084,
 'North Dakota': 762062,
 'Ohio': 11689100,
 'Oklahoma': 3956971,
 'Oregon': 4217737,
 'Pennsylvania': 12801989,
 'Rhode Island': 1059361,
 'South Carolina': 5148714,
 'South Dakota': 884659,
 'Tennessee': 6829174,
 'Texas': 28995881,
 'Utah': 3205958,
 'Vermont': 623989,
 'Virginia': 8535519,
 'Washington': 7614893,
 'West Virginia': 1792147,
 'Wisconsin': 5822434,
 'Wyoming': 578759,
 'Puerto Rico': 3193694,
 'USA': 328239523}


us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'USA':'USA'
}

### Pre-defined functions (modified from hackathon) — testing & vaccination intervention

In [4]:
"""
Function: test_intervention - creates intervention for a testing strategy
Note: this only works for cases generated at event 

To-Do
---------
For testing, esp daily antigen, shouldnt the days_testing argument = 1, instead of 0. We can specify an end date to ensure that testing stops on day 1? 
See documentation: # https://docs.idmod.org/projects/covasim/en/latest/covasim.interventions.html?highlight=test_num#covasim.interventions.test_num

Parameters
----------
test_del: Delay in test result
days_testing: Number of days in event testing susceptibles
subtarg: Indices of population to subtarget
sens: Sensitivity of test used

""" 

def test_intervention(test_del, days_testing, rapid, window, pars, incidence_avg, sens=.984, subtarg=None):
    if (rapid):
        pars['pop_infected'] -= sens * pars['pop_infected']
    else:
        num_preinfectious = sum([((1/window) * pars['pop_size'] * incidence_avg * x) for x in range(1,window+1)])
        
        # old preinfections
        # Newly infected are those infected after test (assume even distribution of test date from 1 to 'window' days ago)
        # Assume 12 days infectious, all tests within 4.6 day period from exposed->infectious (max window 4)
        # These people will test negative even with a perfect test, because they were exposed after testing negative
        # num_preinfectious = prevalence * pars['pop_size'] * sum([x*(1/(12*window)) for x in range(1,window+1)])
        
        # Finally, remove population that is detected by the reported tests
        pars['pop_infected'] += num_preinfectious - sens * pars['pop_infected']
        
    
    return cv.test_num(daily_tests=pars['pop_size']*days_testing, 
                       start_day=pars['start_day'], 
                       subtarget=subtarg,
                       symp_test=0,
                       sensitivity=sens,
                       test_delay=test_del), pars

"""
Function: vaccine_intervention: Creates intervention for a vaccine-based entry strategy. returns intervention

To-Do
---------
Check cummulative infections for mandatory vaxing - values seem way too high?

Parameters
----------
percent_vax: Percentage of overall population vaccinated
passport: If true, 100% of attendees must be vaccinated
efficacy_inf: Efficacy against infection
efficacy_symp: Efficacy against symptoms 
"""


def vaccine_intervention(percent_vax,efficacy_inf, efficacy_symp, pars, passport = False):
    pars['pop_infected'] -= (1-efficacy_inf) * percent_vax * pars['pop_infected']
    if passport:
        percent_vax = 1
    return cv.simple_vaccine(days=0, prob=percent_vax, rel_sus=efficacy_inf, rel_symp=efficacy_symp), pars

In [5]:
# load data files

# ----- State daily data ----- 
can_state = pd.read_csv("states.csv")


# ----- USA timeseries data ----- 
can_usa_timeseries = pd.read_csv("UStimeseries.csv")

can_usa_timeseries.dropna(subset = ['actuals.cases'], inplace=True)
can_usa_timeseries.dropna(subset = ['metrics.vaccinationsCompletedRatio'], inplace=True)

can_usa_timeseries['date']=pd.to_datetime(can_usa_timeseries.date)
can_usa_timeseries.sort_values(by='date', inplace=True)


# ----- State timeseries data ----- 
can_state_timeseries = pd.read_csv("statestimeseries.csv")
can_state_timeseries.dropna(subset = ['actuals.cases'], inplace=True)

can_state_timeseries['date']=pd.to_datetime(can_state_timeseries.date)
can_state_timeseries.sort_values(by='date', inplace=True)

### App simulation 

In [11]:
def run_sim(n_clicks, location, event_duration, num_people, event_setting, test_setting, vax_setting, npi, show_plot=False):
    if n_clicks < 1: 
        avp_fig = go.Figure()
        return avp_fig, avp_fig
    # ----- basic event characteristics ----- 
    event_duration = event_duration if event_duration != None else 1 #temp fix for misfiring nclicks
    num_people = num_people if num_people != None else 1
    covsim_location = "USA-" + location if location != "USA" else "USA"
    event_environment= None if event_setting == "Mixed" else event_setting
    test_type= None if test_setting == "No Testing" else test_setting
    use_vaccines=False if vax_setting == "No Vaccination" else True
    mandatory_vax=True if vax_setting == "Mandatory Vaccination" else False
    mask_wearing = True if 1 in npi else False
    # print(npi)
    # print("test_setting: ", test_setting)
    # print("vax_setting: ", vax_setting)
    # print("event_environment:", event_environment, 
    # "test_type:", test_type, "use_vaccines:", use_vaccines, "mandatory_vax:", mandatory_vax)

    # ---------- Point estimates ---------- 
    indoor_factor = 9.35
    outdoor_factor = .11 
    mask_factor = .56
    ventilation_factor = .69
    capacity_factor = .5
    start_day = '2021-07-01' # default start date
    variant_transmissibility = 1.67 
    
    # location-specific & other characteristics
    under_rep_factor = 4.3
    location_pop = population_dict[location]
    
    state_abv = us_state_abbrev[location]
    if state_abv == 'USA':
        location_total_inf = can_usa_timeseries.iloc[-1]['actuals.cases'] # total cases for a location
        location_cases_d10 = sum(can_usa_timeseries.iloc[range(-10,0,1)]['actuals.newCases'].values) # total cases in last 10 days
        perc_vax = can_usa_timeseries.iloc[-1]['metrics.vaccinationsCompletedRatio'] # % of people fully vaccinated
        incidence_avg = mean(can_usa_timeseries.iloc[range(-7,0,1)]['actuals.newCases'].values) # avg new cases last 7 days
    else:
        location_total_inf = can_state[can_state['state']==state_abv]['actuals.cases'].values[0]  
        location_cases_d10 = sum(can_state_timeseries[can_state_timeseries['state']==state_abv].iloc[range(-10,0,1)]['actuals.newCases'].values)
        perc_vax = can_state[can_state['state']==state_abv]['metrics.vaccinationsCompletedRatio'].values[0]
        incidence_avg = mean(can_state_timeseries[can_state_timeseries['state']==state_abv].iloc[range(-7,0,1)]['actuals.newCases'].values)
        
    # print("location: {}, location abv: {}".format(location, state_abv))
    # print("location_total_inf: {}, location_cases_d10: {}, perc_vax: {}".format(location_total_inf, location_cases_d10, perc_vax))
    
    # calculate location specific prevelance
    prevalence = (location_cases_d10*under_rep_factor)/location_pop # 0.014 - old number
    
    # calculate location specific susceptibility proportion
    susceptibility_proportion = 1-((location_total_inf * under_rep_factor)/location_pop) #0.747 - old number
    
    # vaccination levels:
    v_efficacy_inf = .1 # Vax efficacy against infection (e.g .2 == 80% efficacy)
    v_efficacy_symp = .06 # Vax efficacy against symptoms
    
    # ---------- Define parameters in covasim format ---------- 
    pars = dict(
        pop_type = 'hybrid', # Use a more realistic population model
        pop_size = num_people,
        pop_infected = num_people*prevalence,
        start_day = start_day,
        n_days = event_duration,
        location = covsim_location # Case insensitive
    )
    
    # ----------  model changes (variants, environment, NPIs, testing, vaccines) to beta on day 0 ---------- 
    all_interventions = []
    
    # variant transmissibility
    variant_trans = cv.change_beta(0, variant_transmissibility*susceptibility_proportion)
    all_interventions.append(variant_trans)
    
    # environment
    if event_environment == "Indoor":
        environment = cv.change_beta(0, indoor_factor)
        all_interventions.append(environment)
    elif event_environment == "Outdoor":
        environment = cv.change_beta(0, outdoor_factor)
        all_interventions.append(environment)
    
    # mask wearing
    if mask_wearing:
        masks = cv.change_beta(0, mask_factor)
        all_interventions.append(masks)   
    
    # testing 
    """
    -   test_del: Delay in test result
    -   days_testing: Number of days in event testing susceptibles
    -   subtarg: Indices of population to subtarget
    -   sens: Sensitivity of test used
    """
 
    testing_scenarios = {
        # format = [test_del, days_testing, rapid, window]
        "Entry antigen":[0,0,True,0],
        "Daily antigen": [0,event_duration,True,0], #[0,7,True,0]
        "PCR 2-day":[0,0,False,2],
        "PCR 4-day":[0,0,False,4],        
    } 
    subtarg = None # Subtargetting of tests
    sens = .984 # Sensitivity of test used
    if test_type != None:
        test_int, pars = test_intervention(test_del=testing_scenarios[test_type][0], 
                                           days_testing=testing_scenarios[test_type][1], 
                                           rapid=testing_scenarios[test_type][2], 
                                           window=testing_scenarios[test_type][3], 
                                           pars=pars,
                                           incidence_avg=incidence_avg,
                                           sens=sens)
        all_interventions.append(test_int)
        
    
    # vaccinations - assumes state level vax numbers (default), unless otherwise specified (either mandatory vax, or no vax)
    passport = mandatory_vax # True if 100% of attendees must be vaccinated
    if use_vaccines:
        vc, pars = vaccine_intervention(perc_vax, v_efficacy_inf, v_efficacy_symp, pars, passport)
        all_interventions.append(vc)

    
    # ---------- run simulation ----------
    # print("Running SIMULATION")
    sim = cv.Sim(pars, interventions = all_interventions)
    msim = cv.MultiSim(sim)
    msim.run(n_runs=10)
    msim.mean()
    # msim.plot(to_plot=['new_infections', 'cum_infections'])
    sim_json = msim.to_json()

    # ---------- plotting ----------
    df_new_infections = pd.DataFrame(list(zip(sim_json['results']['t'], 
                            sim_json['results']['new_infections'],
                            sim_json['results']['new_infections_low'],
                            sim_json['results']['new_infections_high'])),
                    columns =['dates','new_infections', 'new_infections_low', 'new_infections_high'])
    
    df_cum_infections = pd.DataFrame(list(zip(sim_json['results']['t'], 
                            sim_json['results']['cum_infections'],
                            sim_json['results']['cum_infections_low'],
                            sim_json['results']['cum_infections_high'])),
                    columns =['dates','cum_infections', 'cum_infections_low', 'cum_infections_high'])

    df_new_infections[df_new_infections < 0] = 0
    df_cum_infections[df_cum_infections < 0] = 0


    avp_fig = go.Figure([
        go.Scatter(
            x=df_new_infections['dates'].tolist(),
            y=df_new_infections['new_infections'].tolist(),
            line=dict(color='rgb(0,100,80)'),
            mode='lines',
            name="Estimated New Cases",
            showlegend=True
        ),
        go.Scatter(
            x=df_new_infections['dates'].tolist()+df_new_infections['dates'].tolist()[::-1], # x, then x reversed
            y=df_new_infections['new_infections_high'].tolist()+df_new_infections['new_infections_low'].tolist()[::-1], # upper, then lower reversed
            fill='toself',
            fillcolor='rgba(0,100,80,0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=True,
            name = '95% Confidence Interval'
        )
    ])
    #  https://stackoverflow.com/questions/55704058/plotly-how-to-set-the-range-of-the-y-axis
    avp_fig.update_layout(yaxis_range=[-0.1,max(max(df_new_infections['new_infections_high'].tolist()), 8)+2],
    yaxis_title='Cases',
    xaxis_title='Day',
    title_text='New Daily Infections', title_x=0.5, title_y=0.875,
    hovermode="x",
    xaxis = dict(dtick = 1),
    showlegend=True,
    legend=dict(yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01)
    )

    

    cum_infections_high = df_cum_infections['cum_infections_high'].tolist()
    cum_infections = df_cum_infections['cum_infections'].tolist()
    loc_fig = go.Figure([
        go.Scatter(
            x=df_cum_infections['dates'].tolist(),
            y=df_cum_infections['cum_infections'].tolist(),
            line=dict(color='rgb(0,100,80)'),
            mode='lines',
            name="Estimated Total Cases",
            showlegend=True
        ),
        go.Scatter(
            x=df_cum_infections['dates'].tolist()+df_cum_infections['dates'].tolist()[::-1], # x, then x reversed
            y=df_cum_infections['cum_infections_high'].tolist()+df_cum_infections['cum_infections_low'].tolist()[::-1], # upper, then lower reversed
            fill='toself',
            fillcolor='rgba(0,100,80,0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            name = '95% Confidence Interval',
            showlegend=True
        
        )
    ])
    loc_fig.update_layout(
    yaxis_range=[max(-0.1,min(cum_infections)-1),max(max(cum_infections_high), 8)+5],
    yaxis_title='Cases',
    xaxis_title='Day',
    title_text='Cummulative Infections', title_x=0.5, title_y=0.875,
    hovermode="x",
    xaxis = dict(dtick = 1),
    showlegend=True,
    legend=dict(yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01)
    )
    if show_plot:
        avp_fig.show()
        loc_fig.show()
    return df_new_infections, df_cum_infections

## Comparing environment without interventions

In [12]:
"""
Parameter defintions
--------------------
- n_clicks: required for app, ignore
- location: "USA", or one of the 50 states or territories (see us_state_abbrev dict keys for full list). Enter full name.
- event_duration: int, 1-7 days
- num_people: int, 1-100000 individuals
- event_setting: "Mixed", "Indoor", "Outdoor"
- test_setting =  "No Testing", "Entry antigen", "Daily antigen", "PCR 2-day" "PCR 4-day"
- vax_setting = "No Vaccination", "Mandatory Vaccination" otherwise Default (assumes regional prevalence)
- npi = [0] (no masking, default), [1] (masking)
- show_plot = False (default)

Currently the above code does not return negative confidence intervals - to obtain commment out the following code above:
    df_new_infections[df_new_infections < 0] = 0
    df_cum_infections[df_cum_infections < 0] = 0
    
    
The function is currently setup to return two data frames for each scenario - df_new_infections, df_cum_infections.
This can be easily changed by modifying the following code above:

    sim_json = msim.to_json() # <-- save output as json (similar to a dict)

    # ---------- plotting ----------
    
    #  access relevant elements from sim_json, convert to list and then pandas dataframe
    df_new_infections = pd.DataFrame(list(zip(sim_json['results']['t'], 
                            sim_json['results']['new_infections'],
                            sim_json['results']['new_infections_low'],
                            sim_json['results']['new_infections_high'])),
                    columns =['dates','new_infections', 'new_infections_low', 'new_infections_high'])
    
    df_cum_infections = pd.DataFrame(list(zip(sim_json['results']['t'], 
                            sim_json['results']['cum_infections'],
                            sim_json['results']['cum_infections_low'],
                            sim_json['results']['cum_infections_high'])),
                    columns =['dates','cum_infections', 'cum_infections_low', 'cum_infections_high'])


"""



default_scen_new_inf, default_scen_cum_inf = run_sim(n_clicks=1, location="USA", 
                                                     event_duration=4, 
                                                     num_people=10000, 
                                                     event_setting="Mixed", 
                                                     test_setting="No Testing", 
                                                     vax_setting="No Vaccination", 
                                                     npi=[0], 
                                                     show_plot=False)

print(default_scen_new_inf)
print(default_scen_cum_inf)

   dates  new_infections  new_infections_low  new_infections_high
0      0             0.0            0.000000             0.000000
1      1             0.1            0.000000             0.700000
2      2             2.6            0.000000             7.515282
3      3            10.6            3.159570            18.040430
4      4            23.8            8.416892            39.183108
   dates  cum_infections  cum_infections_low  cum_infections_high
0      0           129.0          129.000000           129.000000
1      1           129.1          128.500000           129.700000
2      2           131.7          126.879212           136.520788
3      3           142.3          132.021868           152.578132
4      4           166.1          143.838823           188.361177


In [14]:
# Indoor
default_scen_new_inf, default_scen_cum_inf = run_sim(n_clicks=1, location="USA", 
                                                     event_duration=4, 
                                                     num_people=10000, 
                                                     event_setting="Indoor", 
                                                     test_setting="No Testing", 
                                                     vax_setting="No Vaccination", 
                                                     npi=[0], 
                                                     show_plot=False)

print(default_scen_new_inf)
print(default_scen_cum_inf)

   dates  new_infections  new_infections_low  new_infections_high
0      0             0.0            0.000000             0.000000
1      1             0.2            0.000000             1.400000
2      2            22.1            0.000000            64.868680
3      3            94.8           52.801905           136.798095
4      4           194.4          126.752679           262.047321
   dates  cum_infections  cum_infections_low  cum_infections_high
0      0           129.0          129.000000           129.000000
1      1           129.2          128.000000           130.400000
2      2           151.3          108.703756           193.896244
3      3           246.1          192.245010           299.954990
4      4           440.5          348.720917           532.279083


In [15]:
# Outdoor
default_scen_new_inf, default_scen_cum_inf = run_sim(n_clicks=1, location="USA", 
                                                     event_duration=4, 
                                                     num_people=10000, 
                                                     event_setting="Outdoor", 
                                                     test_setting="No Testing", 
                                                     vax_setting="No Vaccination", 
                                                     npi=[0], 
                                                     show_plot=False)

print(default_scen_new_inf)
print(default_scen_cum_inf)

   dates  new_infections  new_infections_low  new_infections_high
0      0             0.0                 0.0             0.000000
1      1             0.1                 0.0             0.700000
2      2             0.2                 0.0             1.000000
3      3             1.5                 0.0             4.069047
4      4             2.7                 0.0             5.673214
   dates  cum_infections  cum_infections_low  cum_infections_high
0      0           129.0          129.000000           129.000000
1      1           129.1          128.500000           129.700000
2      2           129.3          128.383485           130.216515
3      3           130.8          127.727542           133.872458
4      4           133.5          128.746054           138.253946
